# Comparison of Similarity Metrics

This appendix compares the effect of all similarity metric implementations of the textdistance library [[TeDi](./A_References.ipynb#tedi)] for some example strings pair combinations for each feature to be calculated. The comparison is based on the goldstandard data and is the basis for deciding the similarity metrics for each feature to be used.

In [1]:
import textdistance as tedi

## Table of Contents

- [Data Takeover](#Data-Takeover)
- [Functions for Similarty Metrics Analysis](#Functions-for-Similarty-Metrics-Analysis)
- [Similarity Metric Assessments](#Similarity-Metric-Assessments)
    - [century](#century)

## Data Takeover

As a first step, the training data set as a result of chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb) is read. Some sample strings of this data set will be used for the comparison and assessment of the different metrics implementations.

In [2]:
import os
import pandas as pd
import pickle as pk

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

# Restore results so far
df_feature_base = pd.read_pickle(os.path.join(path_goldstandard, 'feature_base_df.pkl'),
                                 compression=None)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head()

,duplicates,century_x,century_y,coordinate_x,coordinate_y,corporate_110_x,corporate_110_y,corporate_710_x,corporate_710_y,edition_x,edition_y,exactDate_x,exactDate_y,format_prefix_x,format_prefix_y,format_postfix_x,format_postfix_y,person_245c_x,person_245c_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,volumes_x,volumes_y
0,1,2009,2009,[],[],,,,,,,2009,2009,bk,bk,020000,020000,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem englischen übersetzt von...,"emma, roman","emma, roman",,,600 s.,600 s.
1,1,2009,2009,[],[],,,,,,,2009,2009,bk,bk,020000,020000,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem engl. übers. von ursula ...,"emma, roman",emma,,,600 s.,600 s.
2,1,2009,2009,[],[],,,,,,,2009,2009,bk,bk,020000,020000,jane austen ; aus dem englischen übersetzt von...,jane austen,"emma, roman","emma, roman",,,600 s.,600 s.
3,1,2009,2009,[],[],,,,,,,2009,2009,bk,bk,020000,020000,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem englischen übersetzt von...,emma,"emma, roman",,,600 s.,600 s.
4,1,2009,2009,[],[],,,,,,,2009,2009,bk,bk,020000,020000,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem engl. übers. von ursula ...,emma,emma,,,600 s.,600 s.


## Functions for Similarty Metrics Analysis

The available metrics algorithms of library [[TeDi](./A_References.ipynb#tedi)] are listed in the dictionary below. The dictionary will help calculating a similarity value for each implemented algorithm of the library.

In [3]:
tedi_algorithms = {
    # Edit based
    'Hamming' : tedi.Hamming(), 'MLIPNS' : tedi.MLIPNS(), 'Levenshtein' : tedi.Levenshtein(),
    'DamerauLevenshtein' : tedi.DamerauLevenshtein(), 'JaroWinkler' : tedi.JaroWinkler(),
    'StrCmp95' : tedi.StrCmp95(), 'NeedlemanWunsch' : tedi.NeedlemanWunsch(), 'Gotoh' : tedi.Gotoh(),
    'SmithWaterman' : tedi.SmithWaterman(),
    # Token based
    'Jaccard' : tedi.Jaccard(), 'Sorensen' : tedi.Sorensen(), 'Tversky' : tedi.Tversky(), 'Overlap' : tedi.Overlap(),
    'Tanimoto' : tedi.Tanimoto(), 'Cosine' : tedi.Cosine(), 'MongeElkan' : tedi.MongeElkan(), 'Bag' : tedi.Bag(),
    # Sequence based
    'LCSSeq' : tedi.LCSSeq(), 'LCSStr' : tedi.LCSStr(), 'RatcliffObershelp' : tedi.RatcliffObershelp(),
    # Compression based
    'ArithNCD' : tedi.ArithNCD(), 'RLENCD' : tedi.RLENCD(), 'BWTRLENCD' : tedi.BWTRLENCD(),
    'SqrtNCD' : tedi.SqrtNCD(), 'EntropyNCD' : tedi.EntropyNCD(), 'BZ2NCD' : tedi.BZ2NCD(),
    'LZMANCD' : tedi.LZMANCD(), 'ZLIBNCD' : tedi.ZLIBNCD(),
    # Phonetic
    'MRA' : tedi.MRA(), 'Editex' : tedi.Editex(),
    # Simple
    'Prefix' : tedi.Prefix(), 'Postfix' : tedi.Postfix(), 'Length' : tedi.Length(), 'Identity' : tedi.Identity(),
    'Matrix' : tedi.Matrix()
}

This appendix uses function $\texttt{.apply}\_\texttt{similarities()}$ that applies the $\texttt{.normalized}\_\texttt{similarity()}$ function of $\texttt{textdistance}$ for each algorithm object available in the library. The function is implemented in the separate code file [data_analysis_funcs.py](./data_analysis_funcs.py) 

In [4]:
import data_analysis_funcs as daf

## Similarity Metric Assessments

This section iterates through all available similarity metrics of library $\texttt{textdistance}$ and calculates the similarity values for a pair of two sample strings of each feature of the model. The calculated similarity values will be analysed visually and an algorithm will be decided to be used in chapter [Feature Matrix Generation](./3_FeatureMatrixGeneration.ipynb). The decision will be based on a visual assessment that is validated with the literature, [[Chri2012](./A_References.ipynb#chri2012)].

In [5]:
# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(tedi_algorithms)+3

number_of_samples = 30

### century

In [6]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'century_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['ArithNCD'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=number_of_samples)

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
JaroWinkler
StrCmp95
NeedlemanWunsch
Gotoh
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,JaroWinkler,StrCmp95,NeedlemanWunsch,Gotoh,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
45,183u,1973,0.25,0.0,0.25,0.25,0.666667,0.666667,0.625,0.625,0.25,0.333333,0.50,0.333333,0.50,-1.584963,0.50,0.06250,0.50,0.50,0.25,0.50,0.00,0.2,2.928932e-01,0.833333,0.888889,0.913043,0.6,0.250000,0.25,0.25,0.00,1.0,0.0,0.0
8,183u,2002,0.00,0.0,0.00,0.00,0.000000,0.000000,0.500,0.500,0.00,0.000000,0.00,0.000000,0.00,-inf,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.2,1.110223e-16,0.500000,0.814815,0.913043,0.6,0.000000,0.00,0.00,0.00,1.0,0.0,0.0
44,183u,1941,0.25,0.0,0.25,0.25,0.500000,0.500000,0.625,0.625,0.25,0.142857,0.25,0.142857,0.25,-2.807355,0.25,0.03125,0.25,0.25,0.25,0.25,0.00,0.2,1.705407e-01,0.698120,0.851852,0.913043,0.6,0.250000,0.25,0.25,0.00,1.0,0.0,0.0
1,183u,2000,0.00,0.0,0.00,0.00,0.000000,0.000000,0.500,0.500,0.00,0.000000,0.00,0.000000,0.00,-inf,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.2,1.110223e-16,0.468546,0.777778,0.913043,0.6,0.000000,0.00,0.00,0.00,1.0,0.0,0.0
10,183u,2012,0.00,0.0,0.00,0.00,0.000000,0.000000,0.500,0.500,0.00,0.142857,0.25,0.142857,0.25,-2.807355,0.25,0.03125,0.25,0.25,0.25,0.25,0.00,0.2,1.464466e-01,0.666667,0.777778,0.913043,0.6,0.000000,0.00,0.00,0.00,1.0,0.0,0.0
22,183u,1843,0.50,1.0,0.50,0.50,0.866667,0.833333,0.750,0.750,0.50,0.600000,0.75,0.600000,0.75,-0.736966,0.75,0.09375,0.75,0.75,0.50,0.75,0.00,0.2,4.393398e-01,0.916667,0.888889,0.913043,0.6,0.500000,0.50,0.50,0.00,1.0,0.0,0.0
62,183u,2015,0.00,0.0,0.00,0.00,0.000000,0.000000,0.500,0.500,0.00,0.142857,0.25,0.142857,0.25,-2.807355,0.25,0.03125,0.25,0.25,0.25,0.25,0.00,0.2,1.464466e-01,0.750000,0.814815,0.913043,0.6,0.000000,0.00,0.00,0.00,1.0,0.0,0.0
42,183u,1890,0.50,1.0,0.50,0.50,0.666667,0.666667,0.750,0.750,0.50,0.333333,0.50,0.333333,0.50,-1.584963,0.50,0.06250,0.50,0.50,0.50,0.50,0.00,0.2,2.928932e-01,0.833333,0.888889,0.913043,0.6,0.500000,0.50,0.50,0.00,1.0,0.0,0.0
20,183u,1982,0.25,0.0,0.25,0.25,0.666667,0.666667,0.625,0.625,0.25,0.333333,0.50,0.333333,0.50,-1.584963,0.50,0.06250,0.50,0.50,0.25,0.50,0.00,0.2,2.928932e-01,0.833333,0.851852,0.913043,0.6,0.250000,0.25,0.25,0.00,1.0,0.0,0.0
49,183u,2001,0.00,0.0,0.00,0.00,0.000000,0.000000,0.500,0.500,0.00,0.142857,0.25,0.142857,0.25,-2.807355,0.25,0.03125,0.25,0.25,0.25,0.25,0.00,0.2,1.464466e-01,0.666667,0.777778,0.913043,0.6,0.000000,0.00,0.00,0.00,1.0,0.0,0.0


Attribute $\texttt{century}$ is a string of four digits or characters. For calculating the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance), each string pair is compared character-wise. A so called edit distance between the string pair is calculated as the sum of all edit operations needed to convert the strings into each other, [[Chri2012](./A_References.ipynb#chri2012)]. The resulting Hamming similarity can be deduced from the edit distance and the length of one string. This can be easily done in the examples of the DataFrame above. The Hamming similarity shall be used for attribute $\texttt{century}$.

The Hamming similarity has one drawbak, though, looking at the goldstandard data. The attribute may be filled with letter 'u' for 'unknown' digits instead of a number. A letter 'u' will result in an edit distance of 1. This is a statement which need not be true for the bibliographic units that the records describe. On the other hand, a pair of strings with a letter 'u' at the same digit, need not have a distance of 0 for the bibliographical units. For this reason, the Hamming similarity will be adapted for the case of existence of letter 'u' in one of the strings of the pair. The Hamming similarity will be increased by a value of 0.125 for each unknown digit in a string and the maximum of unknown digits of the compared pair will be calculated. This algorithm based on the Hamming similarity will be implemented in chapter [Feature Matrix Generation](./3_FeatureMatrixGeneration.ipynb).

### corporate

In [7]:
df_string_pairs = daf.string_pair_list(df_feature_base, 'corporate_710_x')

for algorithm in tedi_algorithms :
    if algorithm not in ['ArithNCD', 'Gotoh'] :
        daf.apply_similarities(df_string_pairs, tedi_algorithms[algorithm], algorithm)

df_string_pairs.sample(n=number_of_samples)

Hamming
MLIPNS
Levenshtein
DamerauLevenshtein
JaroWinkler
StrCmp95
NeedlemanWunsch
SmithWaterman
Jaccard
Sorensen
Tversky
Overlap
Tanimoto
Cosine
MongeElkan
Bag
LCSSeq
LCSStr
RatcliffObershelp
RLENCD
BWTRLENCD
SqrtNCD
EntropyNCD
BZ2NCD
LZMANCD
ZLIBNCD
MRA
Editex
Prefix
Postfix
Length
Identity
Matrix


,str1,str2,Hamming,MLIPNS,Levenshtein,DamerauLevenshtein,JaroWinkler,StrCmp95,NeedlemanWunsch,SmithWaterman,Jaccard,Sorensen,Tversky,Overlap,Tanimoto,Cosine,MongeElkan,Bag,LCSSeq,LCSStr,RatcliffObershelp,RLENCD,BWTRLENCD,SqrtNCD,EntropyNCD,BZ2NCD,LZMANCD,ZLIBNCD,MRA,Editex,Prefix,Postfix,Length,Identity,Matrix
20,schweizerische normen-vereinigung,"interkantonale lehrmittelzentrale (luzern), st...",0.050633,0.0,0.202532,0.202532,0.524246,0.550022,0.310127,0.121212,0.333333,0.500000,0.333333,0.848485,-1.584963,0.548387,0.014233,0.354430,0.227848,0.037975,0.321429,0.0,0.013333,0.323630,0.920527,0.315789,0.473684,0.294118,0.166667,0.291139,0.000000,0.0,0.417722,0.0,0.0
2,schweizerische normen-vereinigung,"metropolitan opera, metropolitan opera, orches...",0.035088,0.0,0.157895,0.157895,0.484350,0.517843,0.368421,0.060606,0.323529,0.488889,0.323529,0.666667,-1.628031,0.507257,0.012397,0.385965,0.228070,0.052632,0.266667,0.0,0.055556,0.340177,0.921173,0.440678,0.581395,0.295455,0.000000,0.228070,0.000000,0.0,0.578947,0.0,0.0
25,schweizerische normen-vereinigung,oper (köln),0.000000,0.0,0.121212,0.121212,0.494949,0.543434,0.227273,0.181818,0.128205,0.227273,0.128205,0.454545,-2.963474,0.262432,0.006887,0.151515,0.121212,0.060606,0.181818,0.0,0.058824,0.158954,0.861120,0.460000,0.538462,0.179487,0.166667,0.242424,0.000000,0.0,0.333333,0.0,0.0
0,schweizerische normen-vereinigung,,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-inf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.029412,0.000000,0.212628,0.000000,0.230769,0.153846,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
18,schweizerische normen-vereinigung,"berliner philharmoniker, deutsche oper (berlin...",0.000000,0.0,0.245283,0.264151,0.587812,0.602563,0.424528,0.242424,0.409836,0.581395,0.409836,0.757576,-1.286881,0.597785,0.012397,0.471698,0.301887,0.094340,0.255814,0.0,0.080000,0.322767,0.906219,0.492308,0.551020,0.327273,0.000000,0.311321,0.000000,0.0,0.622642,0.0,0.0
35,schweizerische normen-vereinigung,schweizerische normen-vereinigung,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.205882,0.585786,1.000000,0.880000,0.897436,0.923077,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.0
19,schweizerische normen-vereinigung,interkantonale lehrmittelzentrale,0.121212,0.0,0.151515,0.212121,0.585859,0.616162,0.560606,0.121212,0.434783,0.606061,0.434783,0.606061,-1.201634,0.606061,0.010560,0.606061,0.333333,0.060606,0.181818,0.0,0.058824,0.348519,0.890142,0.500000,0.589744,0.358974,0.000000,0.272727,0.000000,0.0,1.000000,0.0,0.0
36,schweizerische normen-vereinigung,trägerverein 600 jahre niklaus von flüe 1417-2017,0.061224,0.0,0.204082,0.204082,0.521408,0.556906,0.438776,0.090909,0.344262,0.512195,0.344262,0.636364,-1.538420,0.522233,0.012856,0.428571,0.244898,0.122449,0.268293,0.0,0.040000,0.288518,0.858649,0.350649,0.489362,0.210526,0.000000,0.285714,0.000000,0.0,0.673469,0.0,0.0
29,schweizerische normen-vereinigung,allgemeine geschichtforschende gesellschaft de...,0.036364,0.0,0.218182,0.218182,0.568434,0.587828,0.409091,0.181818,0.419355,0.590909,0.419355,0.787879,-1.253757,0.610288,0.013774,0.472727,0.272727,0.127273,0.159091,0.0,0.115385,0.389132,0.940127,0.531250,0.551020,0.351852,0.000000,0.354545,0.000000,0.0,0.600000,0.0,0.0
31,schweizerische normen-vereinigung,schweiz,0.212121,0.0,0.212121,0.212121,0.842424,0.842424,0.212121,0.000000,0.212121,0.350000,0.212121,1.000000,-2.237039,0.460566,0.008264,0.212121,0.212121,0.212121,0.350000,0.0,0.088235,0.219143,0.807028,0.540000,0.538462,0.307692,0.500000,0.272727,0.212121,0.0,0.212121,0.0,0.0
